<div style="text-align: right" align="right"><i>Peter Norvig, 3 Jan 2020</i></div>

# Spelling Bee Puzzle

The [3 Jan. 2020 edition of the **538 Riddler**](https://fivethirtyeight.com/features/can-you-solve-the-vexing-vexillology/) concerns the popular NYTimes  [**Spelling Bee**](https://www.nytimes.com/puzzles/spelling-bee) puzzle:

> In this game, seven letters are arranged in a **honeycomb** lattice, with one letter in the center. Here’s the lattice from Dec. 24, 2019:
> 
> <img src="https://fivethirtyeight.com/wp-content/uploads/2020/01/Screen-Shot-2019-12-24-at-5.46.55-PM.png?w=1136" width="150">
> 
> The goal is to identify as many words as possible that meet the following criteria:
> 1. The word must be at least four letters long.
> 2. The word must include the central letter.
> 3. The word cannot include any letter beyond the seven given letters.
>
>Note that letters can be repeated. For example,  GAME and AMALGAM are both acceptable words. Four-letter words are worth 1 point each, while five-letter words are worth 5 points, six-letter words are worth 6 points, etc. Words that use all seven letters in the honeycomb are known as **pangrams** and earn 7 bonus points (in addition to the points for the length of the word). So in the above example, MEGAPLEX is worth 8 + 7 = 15 points.
>
> ***Which seven-letter honeycomb results in the highest possible score?*** To be a valid choice of seven letters, no letter can be repeated, it must not contain the letter S (that would be too easy) and there must be at least one pangram.
>
> For consistency, please use [this word list](https://norvig.com/ngrams/enable1.txt) to check your game score.



Since the referenced word list came from [***my*** web site](https://norvig.com/ngrams), I felt  compelled to solve this puzzle.  (Note it is a standard public domain Scrabble® word list that I happen to host a copy of; I didn't curate it, Mendel Cooper and Alan Beale did.) 

I'll show you how I address the problem. First some imports, then we'll work through 10 steps.

In [1]:
from collections import defaultdict
from dataclasses import dataclass
from itertools   import combinations
from typing      import List, Set, Dict, Tuple, Iterable

# 1: Letters, Lettersets, Words, and Pangrams

Let's start by defining the most basic terms:

- **Letter**: the valid letters are uppercase 'A' to 'Z', but not 'S'.
- **Letterset**: the set of distinct letters in a word.
- **Word**: A string of letters.
- **valid word**: a word of at least 4 letters, all valid, and not more than 7 distinct letters.
- **pangram**: a valid word with exactly 7 distinct letters.
- **word list**: a list of valid  words.

In [2]:
letters   = set('ABCDEFGHIJKLMNOPQR' + 'TUVWXYZ')
Letter    = str
Letterset = str
Word      = str 

def letterset(word) -> Letterset:
    """The set of distinct letters in a word."""
    return ''.join(sorted(set(word)))

def is_valid(word) -> bool:
    """Is word 4 or more valid letters and no more than 7 distinct letters?"""
    return len(word) >= 4 and letters.issuperset(word) and len(set(word)) <= 7 

def is_pangram(word) -> bool: return len(set(word)) == 7

def word_list(text: str) -> List[Word]: 
    """All the valid words in a text (uppercased)."""
    return [w for w in text.upper().split() if is_valid(w)]

Here's a mini word list to experiment with:

In [3]:
mini = word_list('amalgam amalgamation cacciatore erotica em game gem gems glam megaplex')
mini

['AMALGAM', 'CACCIATORE', 'EROTICA', 'GAME', 'GLAM', 'MEGAPLEX']

Note that `em` and `gem` are too short, `gems` has an `s`, and `amalgamation` has 8 distinct letters. We're left with six valid words out of the ten candidate words. The pangrams are:

In [4]:
set(filter(is_pangram, mini))

{'CACCIATORE', 'EROTICA', 'MEGAPLEX'}

Why did I choose to represent a `Letterset` as a sorted string (and not a `set`)? Because:
- A `set` can't be the key of a dict.
- A `frozenset` can be a key, and would be a reasonable choice for `Letterset`, but it:
  - Takes up more memory than a `str`.
  - Is verbose and hard to read when debugging: `frozenset({'A', 'G', 'L', 'M'})`
- A `str` of distinct letters in sorted order fixes all these issues.

In [5]:
assert letterset('AMALGAM') == letterset('GLAM')

letterset('AMALGAM')

'AGLM'

# 2: Honeycombs

A honeycomb lattice consists of:
- A set of seven distinct letters
- The one distinguished center letter

In [6]:
@dataclass(frozen=True, order=True)
class Honeycomb:
    """A Honeycomb lattice, with 7 letters, 1 of which is the center."""
    letters: Letterset 
    center:  Letter    

In [7]:
hc = Honeycomb(letterset('MEGAPLEX'), 'G')
hc

Honeycomb(letters='AEGLMPX', center='G')

# 3: Scoring

- The **word score** is 1 point for a 4-letter word, or the word length for longer words, plus 7 bonus points for a pangram.
- The **game score** for a honeycomb is the sum of the word scores for the words that the honeycomb can make. 
- A honeycomb **can make** a word if:
  - the word contains the honeycomb's center, and
  - every letter in the word is in the honeycomb. 

In [8]:
def word_score(word) -> int: 
    """The points for this word, including bonus for pangram."""
    return 1 if len(word) == 4 else (len(word) + 7 * is_pangram(word))

def game_score(honeycomb, wordlist) -> int:
    """The total score for this honeycomb."""
    return sum(word_score(w) for w in wordlist if can_make(honeycomb, w))

def can_make(honeycomb, word) -> bool:
    """Can the honeycomb make this word?"""
    return honeycomb.center in word and all(L in honeycomb.letters for L in word)

The word scores, game score (on `hc`), and makeable words for `mini` are as follows:

In [9]:
{w: word_score(w) for w in mini}

{'AMALGAM': 7,
 'CACCIATORE': 17,
 'EROTICA': 14,
 'GAME': 1,
 'GLAM': 1,
 'MEGAPLEX': 15}

In [10]:
game_score(hc, mini) # 7 + 1 + 1 + 15

24

In [11]:
{w for w in mini if can_make(hc, w)}

{'AMALGAM', 'GAME', 'GLAM', 'MEGAPLEX'}

# 4: Top Honeycomb on Mini Word List

A simple strategy for finding the top (highest-game-score) honeycomb is:
 - Compile a list of all valid candidate honeycombs.
 - For each honeycomb, compute the game score.
 - Return a (score, honeycomb) tuple with the maximum score.

In [12]:
def top_honeycomb(wordlist) -> Tuple[int, Honeycomb]: 
    """Find a (score, honeycomb) tuple with a highest-scoring honeycomb."""
    return max((game_score(h, wordlist), h) 
               for h in candidate_honeycombs(wordlist))

What are the possible candidate honeycombs? We could try all letters in all slots, but that's a lot of honeycombs. Fortunately, we can use the constraint that a valid honeycomb **must make at least one pangram**.  So the letters of any valid honeycomb must ***be*** the letterset of some pangram (and the center can be any of those letters):

In [13]:
def candidate_honeycombs(wordlist) -> List[Honeycomb]:
    """Valid honeycombs have pangram letters, with any center."""
    return [Honeycomb(letters, center) 
            for letters in pangram_lettersets(wordlist)
            for center in letters]

def pangram_lettersets(wordlist) -> Set[Letterset]:
    """All lettersets from the pangram words in wordlist."""
    return {letterset(w) for w in wordlist if is_pangram(w)}

In [14]:
pangram_lettersets(mini)

{'ACEIORT', 'AEGLMPX'}

In [15]:
candidate_honeycombs(mini) # 2×7 of them

[Honeycomb(letters='ACEIORT', center='A'),
 Honeycomb(letters='ACEIORT', center='C'),
 Honeycomb(letters='ACEIORT', center='E'),
 Honeycomb(letters='ACEIORT', center='I'),
 Honeycomb(letters='ACEIORT', center='O'),
 Honeycomb(letters='ACEIORT', center='R'),
 Honeycomb(letters='ACEIORT', center='T'),
 Honeycomb(letters='AEGLMPX', center='A'),
 Honeycomb(letters='AEGLMPX', center='E'),
 Honeycomb(letters='AEGLMPX', center='G'),
 Honeycomb(letters='AEGLMPX', center='L'),
 Honeycomb(letters='AEGLMPX', center='M'),
 Honeycomb(letters='AEGLMPX', center='P'),
 Honeycomb(letters='AEGLMPX', center='X')]

Now we're ready to find the highest-scoring honeycomb with the `mini` word list:

In [16]:
top_honeycomb(mini)

(31, Honeycomb(letters='ACEIORT', center='T'))

**The program works.** But that's just the mini word list. 

# 5: The Full Word List

Here's the full-scale word list, `enable1.txt`:

In [17]:
! [ -e  enable1.txt ] || curl -O http://norvig.com/ngrams/enable1.txt
! head  enable1.txt
! wc -w enable1.txt

aa
aah
aahed
aahing
aahs
aal
aalii
aaliis
aals
aardvark
  172820 enable1.txt


In [18]:
enable1 = word_list(open('enable1.txt').read())

In [19]:
print(f"""Some counts for 'enable1.txt':
{172820:9,d} total words
{len(enable1):9,d} valid Spelling Bee words
{sum(map(is_pangram, enable1)):9,d} pangram words
{len(pangram_lettersets(enable1)):9,d} distinct pangram lettersets
{len(candidate_honeycombs(enable1)):9,d} candidate pangram-containing honeycombs
{25*24*23*22*21*20*19//720:9,d} or 25 × (24 choose 6) possible honeycombs (98% invalid, non-pangram)""")

Some counts for 'enable1.txt':
  172,820 total words
   44,585 valid Spelling Bee words
   14,741 pangram words
    7,986 distinct pangram lettersets
   55,902 candidate pangram-containing honeycombs
3,364,900 or 25 × (24 choose 6) possible honeycombs (98% invalid, non-pangram)


How long will it take to run `top_honeycomb(enable1)`? Most of the computation time is in `game_score`, which is called once for each of the 44,585 valid words, so let's estimate the total time by first checking how long it takes to compute the game score of a single honeycomb:

In [20]:
%timeit game_score(hc, enable1)

8.47 ms ± 41.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Roughly 8 or 9 milliseconds for one honeycomb. For all 55,902 valid honeycombs (in minutes):

In [21]:
.009 * 55902 / 60

8.385299999999999

About 8 or 9 minutes. I could run `top_honeycomb(enable1)`, get a coffee, come back, and declare victory. 

But I think that a puzzle like this deserves a more elegant solution. And I have an idea. 

# 6: Faster Algorithm: Points Table

Here's my idea:

1. Try every pangram letterset, but do some precomputation to make `game_score` much faster:
  - Compute the `letterset` and `word_score` for each word in the word list.
  - Make a table of `{letterset: total_points}` giving the total points of all words with a given letterset. 
  - I call this a **points table**.
  - These calculations are independent of the honeycomb, so are done once, not 55,902  times. 
2. `game_score2` considers every letter subset of a honeycomb, and sums the point table entries. 
  - Every word that a honeycomb can make is formed from a **letter subset** of the honeycomb's 7 letters. 
  - A letter subset must include the center letter, and may include any non-empty subset of the other 6 letters.
  - So there are 2<sup>6</sup> – 1 = 63 valid letter subsets. 
  - Thus, `game_score2` iterates over just 63 letter subsets; much fewer than 44,585 valid words.


Here's the code:

In [22]:
PointsTable = Dict[Letterset, int] # How many total points does a letterset score?

def top_honeycomb2(wordlist) -> Tuple[int, Honeycomb]: 
    """Find a (score, honeycomb) tuple with a highest-scoring honeycomb."""
    points_table = tabulate_points(wordlist)
    return max((game_score2(h, points_table), h) 
               for h in candidate_honeycombs(wordlist))

def tabulate_points(wordlist) -> PointsTable:
    """A table of {letterset: points} from words."""
    table = defaultdict(int)
    for w in wordlist:
        table[letterset(w)] += word_score(w)
    return table

def letter_subsets(honeycomb) -> List[Letterset]:
    """The 63 subsets of the letters in the honeycomb, each including the center letter."""
    return [letters 
            for n in range(2, 8) 
            for letters in map(''.join, combinations(honeycomb.letters, n))
            if honeycomb.center in letters]

def game_score2(honeycomb, points_table) -> int:
    """The total score for this honeycomb, using a points table."""
    return sum(points_table[s] for s in letter_subsets(honeycomb))

Let's get a feel for how this works.  First, a 4-letter honeycomb has 7 letter subsets and a 7-letter honeycomb has 63:

In [23]:
letter_subsets(Honeycomb('ALMG', 'G')) 

['AG', 'LG', 'MG', 'ALG', 'AMG', 'LMG', 'ALMG']

In [24]:
len(letter_subsets(Honeycomb(letterset('MEGAPLEX'), 'G')))

63

Now let's look at the `mini` word list and the points table for it:

In [25]:
points_table = tabulate_points(mini)
mini, points_table

(['AMALGAM', 'CACCIATORE', 'EROTICA', 'GAME', 'GLAM', 'MEGAPLEX'],
 defaultdict(int, {'AGLM': 8, 'ACEIORT': 31, 'AEGM': 1, 'AEGLMPX': 15}))

The letterset `'AGLM'` gets 8 points (7 for AMALGAM and 1 for GLAM).  `'ACEIORT'` gets 31 points (17 for CACCIATORE and 14 for EROTICA). `'AEGM'` gets 1 for GAME and `'AEGLMPX'` gets 15 for MEGAPLEX. The other 59 lettersets have no words, no points.

Let's make sure the new  `top_honeycomb2` function works as well as the old one:

In [26]:
assert top_honeycomb(mini) == top_honeycomb2(mini)

# 7: The Solution

We can now solve the puzzle on the real word list:

In [27]:
%time top_honeycomb2(enable1)

CPU times: user 1.43 s, sys: 4.11 ms, total: 1.44 s
Wall time: 1.44 s


(3898, Honeycomb(letters='AEGINRT', center='R'))

**Wow! 3898 is a high score!** And the whole computation took **less than 2 seconds**!

We can see that `game_score2` is about 400 times faster than `game_score`:

In [28]:
points_table = tabulate_points(enable1)

%timeit game_score2(hc, points_table)

21.3 µs ± 104 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


# 8: Even Faster Algorithm: Branch and Bound

A run time of less than 2 seconds is pretty good! But I think I can do even better.

Consider **JUKEBOX**. It is a pangram, but with  **J**, **K**,  and **X**, it scores poorly, regardless of the center:

In [29]:
honeycombs = [Honeycomb(letterset('JUKEBOX'), C) for C in 'JUKEBOX']

{h: game_score(h, enable1) for h in honeycombs}

{Honeycomb(letters='BEJKOUX', center='J'): 26,
 Honeycomb(letters='BEJKOUX', center='U'): 32,
 Honeycomb(letters='BEJKOUX', center='K'): 26,
 Honeycomb(letters='BEJKOUX', center='E'): 37,
 Honeycomb(letters='BEJKOUX', center='B'): 49,
 Honeycomb(letters='BEJKOUX', center='O'): 39,
 Honeycomb(letters='BEJKOUX', center='X'): 15}

It would be great if we could determine that **JUKEBOX** is not a top honeycomb in one call to `game_score2`, rather than seven calls. My idea:
- Keep track of the top score found so far.
- For each pangram letterset, ask "if we weren't required to use the center letter, what would this letterset score?"
- Check if that score (which is an upper bound of the score using any one center letter) is higher than the top score so far.
- If yes, then try it with all seven centers; if not then discard it without trying any centers.
  - This is called a [**branch and bound**](https://en.wikipedia.org/wiki/Branch_and_bound) algorithm: prune a whole **branch** (of 7 honeycombs) if an upper **bound** can't beat the top score found so far.

*Note*: To represent a honeycomb with no center, I can just use `Honeycomb(letters, '')`. This works because of a quirk of Python:  `game_score2` checks if `honeycomb.center in letters`; normally in Python the expression `e in s` means "*is* `e` *an element of the collection* `s`", but when `s` is a string it means "*is* `e` *a substring of* `s`", and the empty string is a substring of every string. (If I had represented a `Letterset` as a Python `set`, this wouldn't work.)

Thus, I can rewrite `top_honeycomb` in this more efficient form:

In [30]:
def top_honeycomb3(words) -> Tuple[int, Honeycomb]: 
    """Find a (score, honeycomb) tuple with a highest-scoring honeycomb."""
    points_table = tabulate_points(words)
    top_score, top_honeycomb = -1, None
    pangrams = [s for s in points_table if len(s) == 7]
    for p in pangrams:
        if game_score2(Honeycomb(p, ''), points_table) > top_score:
            for center in p:
                honeycomb = Honeycomb(p, center)
                score = game_score2(honeycomb, points_table)
                if score > top_score:
                    top_score, top_honeycomb = score, honeycomb
    return top_score, top_honeycomb

In [31]:
%time top_honeycomb3(enable1)

CPU times: user 309 ms, sys: 1.67 ms, total: 311 ms
Wall time: 310 ms


(3898, Honeycomb(letters='AEGINRT', center='R'))

Awesome! We get the correct answer, and it runs four times faster.

How many honeycombs does `top_honeycomb3` examine? We can use `functools.lru_cache` to make `Honeycomb` keep track:

In [32]:
import functools
Honeycomb = functools.lru_cache(None)(Honeycomb)
top_honeycomb3(enable1)
Honeycomb.cache_info()

CacheInfo(hits=0, misses=8084, maxsize=None, currsize=8084)

This says that `top_honeycomb3`  examined 8,084 honeycombs; an almost 7-fold improvement over the 55,902 examined by `top_honeycomb2`.

# 9: Fancy Report

I'd like to see the actual words that each honeycomb can make, and I'm curious about how the words are divided up by letterset. Here's a function to provide such a report. I remembered that there is a `fill` function in Python (it is in the `textwrap` module) but this report turned out to be a lot more complicated than I anticipated. I guess it is difficult to create a practical extraction and reporting tool. I feel you, [Larry Wall](http://www.wall.org/~larry/).

In [33]:
from textwrap import fill

def report(honeycomb=None, words=enable1):
    """Print stats, words, and word scores for the given honeycomb (or the top
    honeycomb if no honeycomb is given) over the given word list."""
    bins    = group_by(words, key=letterset)
    if honeycomb is None:
        adj = "Top "
        score, honeycomb = top_honeycomb3(words)
    else:
        adj = ""
        score = game_score(honeycomb, words)
    subsets = letter_subsets(honeycomb)
    nwords  = sum(len(bins[s]) for s in subsets)
    print(f'{adj}{honeycomb} scores {Ns(score, "point")} on {Ns(nwords, "word")}',
          f'from a {len(words):,d} word list:\n')
    for s in sorted(subsets, key=lambda s: (-len(s), s)):
        if bins[s]:
            pts = sum(word_score(w) for w in bins[s])
            wcount = Ns(len(bins[s]), "pangram" if is_pangram(s) else "word")
            intro = f'{s:>7} {Ns(pts, "point"):>10} {wcount:>8} '
            words = [f'{w}({word_score(w)})' for w in sorted(bins[s])]
            print(fill(' '.join(words), width=110, 
                       initial_indent=intro, subsequent_indent=' '*8))
            
def Ns(n, noun):
    """A string with `n` followed by the plural or singular of noun:
    Ns(3, 'bear') => '3 bears'; Ns(1, 'world') => '1 world'"""  
    return f"{n:,d} {noun}{' ' if n == 1 else 's'}"

def group_by(items, key):
    "Group items into bins of a dict, each bin keyed by key(item)."
    bins = defaultdict(list)
    for item in items:
        bins[key(item)].append(item)
    return bins

Here are reports for the mini and full word lists:

In [34]:
report(hc, mini)

Honeycomb(letters='AEGLMPX', center='G') scores 24 points on 4 words from a 6 word list:

AEGLMPX  15 points 1 pangram  MEGAPLEX(15)
   AEGM   1 point   1 word  GAME(1)
   AGLM   8 points  2 words AMALGAM(7) GLAM(1)


In [35]:
report(words=enable1)

Top Honeycomb(letters='AEGINRT', center='R') scores 3,898 points on 537 words from a 44,585 word list:

AEGINRT 832 points 50 pangrams AERATING(15) AGGREGATING(18) ARGENTINE(16) ARGENTITE(16) ENTERTAINING(19)
        ENTRAINING(17) ENTREATING(17) GARNIERITE(17) GARTERING(16) GENERATING(17) GNATTIER(15) GRANITE(14)
        GRATINE(14) GRATINEE(15) GRATINEEING(18) GREATENING(17) INGRATE(14) INGRATIATE(17) INTEGRATE(16)
        INTEGRATING(18) INTENERATING(19) INTERAGE(15) INTERGANG(16) INTERREGNA(17) INTREATING(17)
        ITERATING(16) ITINERATING(18) NATTERING(16) RATTENING(16) REAGGREGATING(20) REATTAINING(18)
        REGENERATING(19) REGRANTING(17) REGRATING(16) REINITIATING(19) REINTEGRATE(18) REINTEGRATING(20)
        REITERATING(18) RETAGGING(16) RETAINING(16) RETARGETING(18) RETEARING(16) RETRAINING(17)
        RETREATING(17) TANGERINE(16) TANGIER(14) TARGETING(16) TATTERING(16) TEARING(14) TREATING(15)
 AEGINR 270 points 35 words AGINNER(7) AGREEING(8) ANEARING(8) ANERGIA(7) ANG

# 10: 'S' Words

What if we allowed honeycombs and words to have an 'S' in them? I'll make a new word list, and report on it:

In [36]:
letters.add('S') # Make 'S' a legal letter
enable1s = word_list(open('enable1.txt').read())

report(words=enable1s)

Top Honeycomb(letters='AEINRST', center='E') scores 8,681 points on 1,179 words from a 98,141 word list:

AEINRST 1,381 points 86 pangrams ANESTRI(14) ANTISERA(15) ANTISTRESS(17) ANTSIER(14) ARENITES(15) ARSENITE(15)
        ARSENITES(16) ARTINESS(15) ARTINESSES(17) ATTAINERS(16) ENTERTAINERS(19) ENTERTAINS(17) ENTRAINERS(17)
        ENTRAINS(15) ENTREATIES(17) ERRANTRIES(17) INERTIAS(15) INSTANTER(16) INTENERATES(18) INTERSTATE(17)
        INTERSTATES(18) INTERSTRAIN(18) INTERSTRAINS(19) INTRASTATE(17) INTREATS(15) IRATENESS(16)
        IRATENESSES(18) ITINERANTS(17) ITINERARIES(18) ITINERATES(17) NASTIER(14) NITRATES(15) RAINIEST(15)
        RATANIES(15) RATINES(14) REATTAINS(16) REINITIATES(18) REINSTATE(16) REINSTATES(17) RESINATE(15)
        RESINATES(16) RESISTANT(16) RESISTANTS(17) RESTRAIN(15) RESTRAINER(17) RESTRAINERS(18) RESTRAINS(16)
        RESTRAINT(16) RESTRAINTS(17) RETAINERS(16) RETAINS(14) RETINAS(14) RETIRANTS(16) RETRAINS(15)
        RETSINA(14) RETSINAS(15) SANITAR

Allowing 'S' words more than doubles the score!

Here are the highest-scoring honeycombs (with and without an S) with their stats and mnemonics:

<img src="http://norvig.com/honeycombs.png" width="350">
<center>
   537 words &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; 1,179 words 
    <br>50 pangrams  &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; 86 pangrams
    <br>3,898 points &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; 8,681 points
    <br> &nbsp; RETAINING &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; ENTERTAINERS
</center>

# Summary

This notebook showed how to find the highest-scoring honeycomb, with a baseline and three key ideas:

1. **Brute Force Enumeration**: Compute game score for every honeycomb; return the highest-scoring.
2. **Pangram Lettersets**: Compute game score for  honeycombs that are pangram lettersets (with all 7 centers).
3. **Points Table**: Precompute score for each letterset; for each candidate honeycomb, sum  63 letter subset scores.
4. **Branch and Bound**: Try all 7 centers only for lettersets that score better than the top score so far.

The key ideas paid off in efficiency improvements:


|Approach|Honeycombs|Reduction|`game_score` Time|Speedup|Overall  Time|Overall Speedup|
|--------|----------|--------|----|---|---|---|
|1. **Brute Force Enumeration**|3,364,900|——|9000 microseconds|——|8.5 hours (est.)|——|
|2. **Pangram Lettersets**|55,902|60×|9000 microseconds|——|500 sec (est.)|60×|
|3. **Points Table**|55,902|——|22 microseconds|400×|1.5 seconds|20,000×|
|4. **Branch and Bound**|8,084 |7×|22 microseconds|——|0.31 seconds|100,000×|

